## Inferential Role Semantics for Natural Language

This is an example notebook that illustrates how to use recursive neural networks to generate and manipulate inferential roles for natural language expressions. The basic idea is to use one neural network to encode a sentence into an embedding, and then use another neural network to decode the sentence's inferential consequences from this embedding.

First, we'll define some functions to do some basic preprocessing on the SNLI dataset.

In [2]:
import enchant 
import random
import pickle
import numpy as np

from collections import namedtuple
from pysem.corpora import SNLI
from pysem.networks import DependencyNetwork
from pysem.generatives import EmbeddingGenerator, EncoderDecoder

checker = enchant.Dict('en_US')
TrainingPair = namedtuple('TrainingPair', ['sentence1', 'sentence2', 'label'])

snli = SNLI('path-to-snli-dataset') # modify this for your local machine
snli.load_xy_pairs()

def repair(sen):
    tokens = DependencyNetwork.parser(sen)
    if len(tokens) > 15:
        return None
    for token in tokens:
        if not checker.check(token.text):
            return None
    return sen

def clean_data(data):
    clean = []
    for item in data:
        s1 = repair(item.sentence1)
        s2 = repair(item.sentence2)
        if s1 == None or s2 == None:
            continue
        else:
            clean.append(TrainingPair(s1, s2, item.label))
    return clean

def build_vocab(data):
    vocab = set()
    for item in data:
        parse1 = DependencyNetwork.parser(item.sentence1)
        parse2 = DependencyNetwork.parser(item.sentence2)
        
        for p in parse1:
            if p.text not in vocab:
                vocab.add(p.text)
        
        for p in parse2:
            if p.text not in vocab:
                vocab.add(p.text)

    return sorted(list(vocab))

In [3]:
clean_dev = clean_data(snli.dev_data)
clean_train = clean_data(snli.train_data)
clean_test = clean_data(snli.test_data)

Next, we'll build a vocab from the set of cleaned sentence pairs. The number of items in the vocab can vary slightly depending on which version of the SpaCy dependency parser is being used.

In [4]:
data = clean_dev + clean_test + clean_train
vocab = build_vocab(data)

In [5]:
print(len(vocab))

22495


Now we can collect all of the sentence pairs standing in entailment relations to one another.

In [10]:
train_data = [d for d in clean_train if d.label == 'entailment']
test_data = [d for d in clean_test if d.label == 'entailment']
dev_data = [d for d in clean_dev if d.label == 'entailment']

print(len(train_data))
print(len(test_data))
print(len(dev_data))

106246
1666
1700


To train a model on the example entailment pairs from SNLI, we can do the following:

In [ ]:
dim = 300

with open('depdict', 'rb') as pfile:
    subvocabs = pickle.load(pfile) # dependency-specific vocabs to include relevant words for a particular POS.

encoder = DependencyNetwork(dim=dim, vocab=snli.vocab)
decoder = EmbeddingGenerator(dim=dim, subvocabs=subvocabs)

learned_model = EncoderDecoder(encoder=encoder, decoder=decoder, data=train_data)
learned_model.train(iters=100, rate=0.001, batchsize=10000)

This is slow, so we can also load model paramters that have been previously generated:

In [11]:
model = EncoderDecoder(encoder=None, decoder=None, data=train_data)
model.load('enc_model_0006_alt.pickle','dec_model_0006_alt.pickle')

In [492]:
sample = random.choice(test_data)

print(sample)

model.encode(sample.sentence1)
model.decode(sample.sentence2)

TrainingPair(sentence1='Little boy in a green sweatshirt playing with his toy train.', sentence2='A boy is playing with toys.', label='entailment')


'a boy is playing with toy .'

In [13]:
def compute_accuracy(data, model):
    total = 0 
    correct = 0

    for item in data:
        model.encoder.forward_pass(item.sentence1)
        model.decoder.forward_pass(item.sentence2, model.encoder.get_root_embedding())

        for node in model.decoder.tree:
            total += 1
            if node.pword.lower() == node.lower_:
                correct += 1

    return float(correct / total)

print(compute_accuracy(train_data, model))
print(compute_accuracy(dev_data, model))

0.7056433181328429
0.6013361169102296


## Simple Entailment Generation Examples

This small amount of data probably isn't enough to generalize outside of the training set, so we'll first check how well the learned decoder is able to generate the entailments it has been trained on.

In [503]:
batch = random.sample(train_data, 5)

for sample in batch:
    model.encode(sample.sentence1)

    print('Sentence: ', sample.sentence1)
    print('Actual Entailment: ', sample.sentence2)
    print('Predicted Entailment: ', model.decode(sample.sentence2))
    print('')

Sentence:  A boy smiles at the camera.
Actual Entailment:  The boy is smiling.
Predicted Entailment:  a boy is smiling .

Sentence:  A firefighter is standing in his uniform holding a hose.
Actual Entailment:  A fireman holds a hose.
Predicted Entailment:  a fireman holds a hose .

Sentence:  A professional orchestra is playing in a dimly lit room.
Actual Entailment:  The people are indoors.
Predicted Entailment:  a group playing indoors .

Sentence:  A person siting against a wall with a dog.
Actual Entailment:  A person sitting.
Predicted Entailment:  a person sitting .

Sentence:  A man is loaded on to a train in a wheelchair.
Actual Entailment:  A man in a wheelchair rides the train.
Predicted Entailment:  a man in a wheelchair is a train .



In [463]:
# s5 = "A man is wearing his shorts"
# s6 = "A crow is below a foot"
# s7 = "One person is eating"
# s8 = "People are getting their picture taken"
# s9 = "A man pulls his cart"

dec_tree = random.sample(train_data, 1).pop()
dec_tree = dec_tree.sentence2

sen = "The young man in colorful shorts is barefoot."
model.encode(sen)
print('Sentence: ', sen)
print('Predicted Entailment: ', model.decode(dec_tree))
print('')

sen = "A young man sleeping next to a dog."
model.encode(sen)
print('Sentence: ', sen)
print('Predicted Entailment: ', model.decode(dec_tree))
print('')

sen = "The 3 dogs are cruising down the street."
model.encode(sen)
print('Sentence: ', sen)
print('Predicted Entailment: ', model.decode(dec_tree))
print('')

sen = "Woman reading a book with a grocery tote."
model.encode(sen)
print('Sentence: ', sen)
print('Predicted Entailment: ', model.decode(dec_tree))
print('')

sen = "A man laughing while at a restaurant."
model.encode(sen)
print('Sentence: ', sen)
print('Predicted Entailment: ', model.decode(dec_tree))
print('')

sen = "Two individuals use a photo kiosk."
model.encode(sen)
print('Sentence: ', sen)
print('Predicted Entailment: ', model.decode(dec_tree))
print('')

sen = "A man pulling items on a cart."
model.encode(sen)
print('Sentence: ', sen)
print('Predicted Entailment: ', model.decode(dec_tree))
print('')

sen = "Three people are riding a carriage pulled by four horses."
model.encode(sen)
print('Sentence: ', sen)
print('Predicted Entailment: ', model.decode(dec_tree))
print('')

Sentence:  The young man in colorful shorts is barefoot.
Predicted Entailment:  the man wearing in the shorts

Sentence:  A young man sleeping next to a dog.
Predicted Entailment:  a man sleeping near a dog

Sentence:  The 3 dogs are cruising down the street.
Predicted Entailment:  the dogs are on the street

Sentence:  Woman reading a book with a grocery tote.
Predicted Entailment:  a woman reading with a book

Sentence:  A man laughing while at a restaurant.
Predicted Entailment:  a man laughing at a restaurant

Sentence:  Two individuals use a photo kiosk.
Predicted Entailment:  the people are at a kiosk

Sentence:  A man pulling items on a cart.
Predicted Entailment:  a man pulling on a cart

Sentence:  Three people are riding a carriage pulled by four horses.
Predicted Entailment:  a horses riding with a carriage



## Random Entailment Generation Examples

We can also generate entailments using randomly chosen trees for the decoding network structure. This doesn't  always work very well.

In [29]:
batch = random.sample(train_data, 5)

for sample in batch:
    model.encode(sample.sentence1)

    print('Sentence: ', sample.sentence1)
    print('Actual Entailment: ', sample.sentence2)
    print('Predicted Entailment: ', model.decode(sample.sentence2))
    print('Random Tree Entailment: ', model.decode())
    print('')

Sentence:  A man looks at a woman and she smiles.
Actual Entailment:  A human looking
Predicted Entailment:  a human smiling
Random Tree Entailment:  a man is smiling her smiles .

Sentence:  Two men dressed in jeans and coats walking down the street with messenger bags.
Actual Entailment:  The men are carrying bags
Predicted Entailment:  the men are wearing bags
Random Tree Entailment:  wearing walking bags .

Sentence:  A man drives a passenger in a yellow semi.
Actual Entailment:  The man is in a car.
Predicted Entailment:  a man driving in a car .
Random Tree Entailment:  a man is driving .

Sentence:  A British royal guard on patrol outside a stone building.
Actual Entailment:  A guard patrols a building.
Predicted Entailment:  a guard is a outdoors .
Random Tree Entailment:  a guard is is outdoors

Sentence:  Men playing a sport during a sunny day.
Actual Entailment:  Men are outside on a sunny day.
Predicted Entailment:  men playing outside in a sunny day .
Random Tree Entailmen

## Generating Entailment Chains (i.e. Inferential Roles)

We can also generate entailment chains by re-encoding a generated sentence, and then generating new sentence from the subsequent encoding. This is kind of neat because it allows us to distill what the model has learned in a network of inferential relationships between sentences. Philosophers sometimes argue that the meaning of sentences is determined by its role or location in such a network.

In [504]:
s1 = 'A black dog with a blue collar is jumping into the water.'
s2 = 'Two police officers are sitting on motorcycles in the road.'
s3 = 'Five people are playing in a gymnasium.'
s4 = 'A man curls up in a blanket on the street.'

sentences = [s1, s2, s3, s4]

for sentence in sentences:
    print('Sentence: ', sentence)
    model.encode(sentence)
    entailment = model.decode()
    print('Predicted Entailment: ', entailment)
    model.encode(entailment)
    print('Next Entailment: ', model.decode())
    print('')

Sentence:  A black dog with a blue collar is jumping into the water.
Predicted Entailment:  there jumping dog jumping a swim .
Next Entailment:  a dog jumping outside .

Sentence:  Two police officers are sitting on motorcycles in the road.
Predicted Entailment:  officers are are to go a motorcycles .
Next Entailment:  officers are are at police motorcycles .

Sentence:  Five people are playing in a gymnasium.
Predicted Entailment:  the people are in gym
Next Entailment:  a are practicing gym

Sentence:  A man curls up in a blanket on the street.
Predicted Entailment:  man is is .
Next Entailment:  a man is in the ground in his ground .



## Substitional Analysis

It is also possible to examine the effect a given word or phrase has on entailment generation via substitutions. Essentially, this involves looking at the difference made to the most likely entailment when a given word or phrase in the input sentence is replaced with another word or phrase.

In [507]:
# we'll use these sentences to generate decoding trees
s2 = 'the dog is walking on her phone'
s3 = 'the dog is outside'
s4 = 'the dog is selling the bone'
s5 = 'a dog wearing some clothes is indoors'
s6 = 'a dog is inside a car'
s7 = 'the dog is furry'
s8 = 'two dogs are alone'
s9 = 'The dog is not outdoors'

def substitution(model, sentence1, sentence2):
    model.encode(sentence1)

    print('Sentence: ', sentence1)
    print('Predicted Entailment: ', model.decode(sentence2))
    print('')    

s1 = 'A boy in a beige shirt is sleeping in a car.'
substitution(model, s1, s2)
    
s1 = 'A girl in a beige shirt is sleeping in a car.'
substitution(model, s1, s2)

s1 = 'A man in a beige shirt is sleeping in a car.'
substitution(model, s1, s2)

s1 = 'A woman in a beige shirt is sleeping in a car.'
substitution(model, s1, s2)

s1 = 'A boy in a beige shirt is sleeping in a car.'
substitution(model, s1, s3) 

s1 = 'A woman in a beige shirt is sleeping in a car.'
substitution(model, s1, s3)

s1 = 'A man in a beige shirt is driving in a car.'
substitution(model, s1, s4)

s1 = 'A person in a beige shirt is selling her car.'
substitution(model, s1, s4)

s1 = 'A boy in a red shirt is waiting in a store.'
substitution(model, s1, s5)

s1 = 'Some men in red shirts are waiting in a store.'
substitution(model, s1, s6)

s1 = 'Many women in red shirts are waiting in a store.'
substitution(model, s1, s6)

s1 = 'A girl and a boy are waiting inside a store.'
substitution(model, s1, s8)

s1 = 'A girl and a boy are waiting inside a park.'
substitution(model, s1, s8)

s1 = 'A boy is in the car.'
substitution(model, s1, s9)

s1 = 'A boy is in the store.'
substitution(model, s1, s9)

Sentence:  A boy in a beige shirt is sleeping in a car.
Predicted Entailment:  a boy is sleeping in his car

Sentence:  A girl in a beige shirt is sleeping in a car.
Predicted Entailment:  a girl is sleeping in her car

Sentence:  A man in a beige shirt is sleeping in a car.
Predicted Entailment:  a man is sleeping in his car

Sentence:  A woman in a beige shirt is sleeping in a car.
Predicted Entailment:  a woman is sleeping in her car

Sentence:  A boy in a beige shirt is sleeping in a car.
Predicted Entailment:  a boy sleeping indoors

Sentence:  A woman in a beige shirt is sleeping in a car.
Predicted Entailment:  a woman sleeping inside

Sentence:  A man in a beige shirt is driving in a car.
Predicted Entailment:  a man is driving a car

Sentence:  A person in a beige shirt is selling her car.
Predicted Entailment:  a person is selling a car

Sentence:  A boy in a red shirt is waiting in a store.
Predicted Entailment:  a boy wearing a shirt is indoors

Sentence:  Some men in red s

## Mapping Multiple Sentences to a Common Description:

Here we can draw inferences that connect a group of sentences to single sentence that they all entail.

In [508]:
s1 = 'A fisherman using a cellphone on a boat.'
s2 = 'A man is on the street'
substitution(model, s1, s2)

s1 = 'A Man is eating food next to a child on a bench.'
s2 = 'A man is on the street'
substitution(model, s1, s2)

s1 = 'A shirtless man skateboards on a ledge.'
s2 = 'A man is on the street'
substitution(model, s1, s2)

s1 = 'A man wearing a hat and boots is digging for something in the snow.'
s2 = 'A man is on the street'
substitution(model, s1, s2)

s1 = 'A man is on a boat.'
s2 = 'A man is outside'
substitution(model, s1, s2)

s1 = 'A man is on a bench.'
s2 = 'A man is outside'
substitution(model, s1, s2)

s1 = 'A man is on a skateboard.'
s2 = 'A man is outside'
substitution(model, s1, s2)

s1 = 'A man is in the snow.'
s2 = 'A man is outside'
substitution(model, s1, s2)


Sentence:  A fisherman using a cellphone on a boat.
Predicted Entailment:  a person is on a boat

Sentence:  A Man is eating food next to a child on a bench.
Predicted Entailment:  a man is on a bench

Sentence:  A shirtless man skateboards on a ledge.
Predicted Entailment:  a man is on a skateboard

Sentence:  A man wearing a hat and boots is digging for something in the snow.
Predicted Entailment:  a man digging in the snow

Sentence:  A man is on a boat.
Predicted Entailment:  a man is outside

Sentence:  A man is on a bench.
Predicted Entailment:  a man is outside

Sentence:  A man is on a skateboard.
Predicted Entailment:  a man is outside

Sentence:  A man is in the snow.
Predicted Entailment:  a man is outside



Here's another example of a building out an inferentail role using a single starting sentence:

In [509]:
s1 = 'Some kids are wrestling on an inflatable raft.'
s2 = 'the boy is on the beach.'
substitution(model, s1, s2)

s2 = 'the kids are outside.'
substitution(model, s1, s2)

s2 = 'Some kids wrestle outside in the sun.'
substitution(model, s1, s2)

s2 = 'The kids are with an inflatable raft.'
substitution(model, s1, s2)

s2 = 'The kids wrestle together.'
substitution(model, s1, s2)

s2 = 'young kids wrestle with each other.'
substitution(model, s1, s2)

s2 = 'old children play all over the water.'
substitution(model, s1, s2)

s2 = 'Some kids are with each other.'
substitution(model, s1, s2)

s2 = 'The kids play on a raft under the water.'
substitution(model, s1, s2)

s1 = 'Several kids are around on a raft.'
substitution(model, s1, s2)

s2 = 'They raft on three kids.'
substitution(model, s1, s2)

s2 = 'a rafts used in the match.'
substitution(model, s1, s2)

s2 = 'at least two kids are outside.'
substitution(model, s1, s2)

s1 = 'Some kids are around.'
substitution(model, s1, s2)

s2 = 'More than one kid is wet.'
substitution(model, s1, s2)

s2 = 'Those kids are not very pleased.'
substitution(model, s1, s2)

Sentence:  Some kids are wrestling on an inflatable raft.
Predicted Entailment:  some kids are on a raft .

Sentence:  Some kids are wrestling on an inflatable raft.
Predicted Entailment:  some kids are around .

Sentence:  Some kids are wrestling on an inflatable raft.
Predicted Entailment:  some kids are around on a raft .

Sentence:  Some kids are wrestling on an inflatable raft.
Predicted Entailment:  some kids are on a inflatable raft .

Sentence:  Some kids are wrestling on an inflatable raft.
Predicted Entailment:  some kids are around .

Sentence:  Some kids are wrestling on an inflatable raft.
Predicted Entailment:  several kids are on a raft .

Sentence:  Some kids are wrestling on an inflatable raft.
Predicted Entailment:  several kids are around on a raft .

Sentence:  Some kids are wrestling on an inflatable raft.
Predicted Entailment:  some kids are on a raft .

Sentence:  Some kids are wrestling on an inflatable raft.
Predicted Entailment:  some kids are on a raft of the

## Conditioned Inferences

It is also possible to constrain the decoding process to selectively navigate the inferentail role associated with a particular linguistic expression.

In [510]:
def condition(model, s1, s2, condition, sen=None):
    if sen: 
        model.encoder.forward_pass(condition)
        cond = model.encoder.get_root_embedding()
    else:
        cond = model.encoder.vectors[condition]
    
    model.encode(s1)
    model.decoder.forward_pass(s2, model.encoder.get_root_embedding() + cond)

    predicted = [node.pword for node in model.decoder.tree]
    print('Sentence: ', s1)
    print('Conditioning Context: ', condition)
    print('Predicted Entailment: ', ' '.join(predicted))
    print('')
      
s1 = 'A person wearing a red shirt is falling off a white surfboard.'
s2 = 'A person is falling into the water.'
cond_word = 'surf'
condition(model, s1, s2, cond_word)
       
cond_word = 'ocean'
condition(model, s1, s2, cond_word)

cond_word = 'swim'
condition(model, s1, s2, cond_word)

cond_word = 'fall'
condition(model, s1, s2, cond_word)

cond_word = 'white'
condition(model, s1, s2, cond_word)

s1 = "A man is steering his ship out at sea."
s2 = "A man sleeps in the ocean."
cond_word = 'water'
condition(model, s1, s2, cond_word)
       
cond_word = 'fish'
condition(model, s1, s2, cond_word)

s2 = "A man sleeps in the ocean."
cond_word = 'sails'
condition(model, s1, s2, cond_word)

cond_word = 'steering'
condition(model, s1, s2, cond_word)

cond_word = 'voyage'
condition(model, s1, s2, cond_word)

cond_word = 'sea'
condition(model, s1, s2, cond_word)

s1 = 'A mother and daughter walk along the side of a bridge.'
s2 = 'Two people are walking.'
cond_sen = 'How many people are walking?'
condition(model, s1, s2, cond_sen, sen=True)

s1 = 'A mother and daughter walk along the side of a bridge.'
s2 = 'The mother and daughter walk together.'
cond_sen = 'Are the mother and daughter walking?'
condition(model, s1, s2, cond_sen, sen=True)

s1 = 'A mother and daughter walk along the side of a bridge.'
s2 = 'They are above the water.'
cond_sen = 'What are the mother and daughter doing?'
condition(model, s1, s2, cond_sen, sen=True)

s1 = 'A mother and daughter walk along the side of a bridge.'
s2 = 'This bridge is quite tall.'
cond_sen = 'How tall is the bridge?'
condition(model, s1, s2, cond_sen, sen=True)

s1 = 'A mother and daughter walk along the side of a bridge.'
s2 = 'Two people are together'
cond_sen = 'Are two people with one another?'
condition(model, s1, s2, cond_sen, sen=True)

s1 = 'A mother and daughter walk along the side of a bridge.'
s2 = 'mother is taller.'
cond_sen = 'Who is taller?'
condition(model, s1, s2, cond_sen, sen=True)

Sentence:  A person wearing a red shirt is falling off a white surfboard.
Conditioning Context:  surf
Predicted Entailment:  a surfer is surfing on a surfboard .

Sentence:  A person wearing a red shirt is falling off a white surfboard.
Conditioning Context:  ocean
Predicted Entailment:  a person is is on the ocean .

Sentence:  A person wearing a red shirt is falling off a white surfboard.
Conditioning Context:  swim
Predicted Entailment:  a person is swim off a surfboard .

Sentence:  A person wearing a red shirt is falling off a white surfboard.
Conditioning Context:  fall
Predicted Entailment:  a person is falls off a air .

Sentence:  A person wearing a red shirt is falling off a white surfboard.
Conditioning Context:  white
Predicted Entailment:  a person is wearing off a surfboard .

Sentence:  A man is steering his ship out at sea.
Conditioning Context:  water
Predicted Entailment:  a man is in the water .

Sentence:  A man is steering his ship out at sea.
Conditioning Context: